In [1]:
#!pip install pytrends
from pytrends.request import TrendReq
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd

In [37]:
def retrieve_data(lista: list, mode):
    pytrends = TrendReq()
    if mode == 'overall':
        pytrends.build_payload(kw_list=lista, timeframe='2021-01-01 2023-02-21', geo='IT')
        dataframe = pytrends.interest_over_time().iloc[:,:-1]

    elif mode == 'individual':
        dataframe = pd.DataFrame()
        for i in lista:
            pytrends.build_payload(kw_list=[i], timeframe='2021-01-01 2023-02-21', geo='IT')
            dataframe = pd.concat([dataframe, pytrends.interest_over_time().iloc[:,:-1]], axis = 1)

    dataframe.reset_index(inplace = True)
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe.set_index('date', inplace = True)
    return dataframe

topic = ['putin', 'zelensky']

data_individual = retrieve_data(topic, 'individual')
data_overall = retrieve_data(topic, 'overall')